<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_NRPy`: Source Terms

## Author: Patrick Nelson

<a id='intro'></a>

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code produces the expected results for generated functions.

## This module presents the functionality of [GiRaFFE_NRPy_Source_Terms.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Source_Terms.py).

## Introduction: 
This writes and documents the C code that `GiRaFFE_NRPy` uses to compute the source terms for the right-hand sides of the evolution equations for the unstaggered prescription.

The equations themselves are already coded up in other functions; however, for the $\tilde{S}_i$ source term, we will need derivatives of the metric. It will be most efficient and accurate to take them using the interpolated metric values that we will have calculated anyway; however, we will need to write our derivatives in a nonstandard way within NRPy+ in order to take advantage of this, writing our own code for memory access.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#stilde_source): The $\tilde{S}_i$ source term
1. [Step 3](#code_validation): Code Validation against original C code
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import cmdline_helper as cmd
outdir = os.path.join("GiRaFFE_NRPy/GiRaFFE_Ccode_validation/RHSs/")
cmd.mkdir(outdir)

<a id='stilde_source'></a>

## Step 1: The $\tilde{S}_i$ source term \[Back to [top](#toc)\]
$$\label{stilde_source}$$

We start in the usual way - import the modules we need. We will also import the Levi-Civita symbol from `indexedexp.py` and use it to set the Levi-Civita tensor $\epsilon^{ijk} = [ijk]/\sqrt{\gamma}$.

In [5]:
# Step 1: The StildeD RHS *source* term
from outputC import outCfunction, lhrh # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import GRHD.equations as GRHD    # NRPy+: Generate general relativistic hydrodynamics equations

thismodule = "GiRaFFE_NRPy_Source_Terms"

# Declare all the Cparameters we will need
metricderivDDD = ixp.declarerank3("metricderivDDD","sym01",DIM=3)
shiftderivUD = ixp.declarerank2("shiftderivUD","nosym",DIM=3)
lapsederivD = ixp.declarerank1("lapsederivD",DIM=3)

general_access = """const REAL gammaDD00 = auxevol_gfs[IDX4S(GAMMADD00GF,i0,i1,i2)];
const REAL gammaDD01 = auxevol_gfs[IDX4S(GAMMADD01GF,i0,i1,i2)];
const REAL gammaDD02 = auxevol_gfs[IDX4S(GAMMADD02GF,i0,i1,i2)];
const REAL gammaDD11 = auxevol_gfs[IDX4S(GAMMADD11GF,i0,i1,i2)];
const REAL gammaDD12 = auxevol_gfs[IDX4S(GAMMADD12GF,i0,i1,i2)];
const REAL gammaDD22 = auxevol_gfs[IDX4S(GAMMADD22GF,i0,i1,i2)];
const REAL betaU0 = auxevol_gfs[IDX4S(BETAU0GF,i0,i1,i2)];
const REAL betaU1 = auxevol_gfs[IDX4S(BETAU1GF,i0,i1,i2)];
const REAL betaU2 = auxevol_gfs[IDX4S(BETAU2GF,i0,i1,i2)];
const REAL alpha = auxevol_gfs[IDX4S(ALPHAGF,i0,i1,i2)];
const REAL ValenciavU0 = auxevol_gfs[IDX4S(VALENCIAVU0GF,i0,i1,i2)];
const REAL ValenciavU1 = auxevol_gfs[IDX4S(VALENCIAVU1GF,i0,i1,i2)];
const REAL ValenciavU2 = auxevol_gfs[IDX4S(VALENCIAVU2GF,i0,i1,i2)];
const REAL BU0 = auxevol_gfs[IDX4S(BU0GF,i0,i1,i2)];
const REAL BU1 = auxevol_gfs[IDX4S(BU1GF,i0,i1,i2)];
const REAL BU2 = auxevol_gfs[IDX4S(BU2GF,i0,i1,i2)];
"""

def idxp1(dirn):
    if dirn=="0":
        return "i0+1,i1,i2"
    if dirn=="1":
        return "i0,i1+1,i2"
    if dirn=="2":
        return "i0,i1,i2+1"

# Next we evaluate needed derivatives of the metric, based on their values at cell faces
metric_deriv_access = []
for dirn in ["0","1","2"]:
    metric_deriv_access.append("")
    for var in ["GAMMA_FACEDD00", "GAMMA_FACEDD01", "GAMMA_FACEDD02",
                "GAMMA_FACEDD11", "GAMMA_FACEDD12", "GAMMA_FACEDD22",
                "BETA_FACEU0", "BETA_FACEU1", "BETA_FACEU2","ALPHA_FACEGF"]:
        lhsvar = var.lower().replace("dd","DD").replace("u","U").replace("_face","")
        # e.g.,
        # const REAL gammaDD00dD0 = (auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1,i2)])/dxx0;
        metric_deriv_access[dirn] += "const REAL "+lhsvar+"dD"+dirn+" = (auxevol_gfs[IDX4S("+var+"GF,"+idxp1(dirn)+")]-auxevol_gfs[IDX4S("+var+"GF,i0,i1,i2)])/dxx"+dirn+";\n"
    metric_deriv_access[dirn] += "REAL Stilde_rhsD"+dirn+";\n"

print(metric_deriv_access[0])

write_final_quantity = ixp.zerorank1(DIM=3)
write_final_quantity[0] = """rhs_gfs[IDX4S(STILDED0GF,i0,i1,i2)] += Stilde_rhsD0;
"""
write_final_quantity[1] = """rhs_gfs[IDX4S(STILDED1GF,i0,i1,i2)] += Stilde_rhsD1;
"""
write_final_quantity[2] = """rhs_gfs[IDX4S(STILDED2GF,i0,i1,i2)] += Stilde_rhsD2;
"""

# Declare this symbol:
sqrt4pi = par.Cparameters("REAL",thismodule,"sqrt4pi","sqrt(4.0*M_PI)")

def write_out_functions_for_StildeD_source_term(gammaDD)
# We need to rerun a few of these functions with the reset lists to make sure these functions
# don't cheat by using analytic expressions
GRHD.compute_sqrtgammaDET(gammaDD)
GRHD.u4U_in_terms_of_ValenciavU__rescale_ValenciavU_by_applying_speed_limit(alpha, betaU, gammaDD, ValenciavU)
GRFFE.compute_smallb4U(gammaDD, betaU, alpha, GRHD.u4U_ito_ValenciavU, BU, sqrt4pi)
GRFFE.compute_smallbsquared(gammaDD, betaU, alpha, GRFFE.smallb4U)
GRFFE.compute_TEM4UU(gammaDD,betaU,alpha, GRFFE.smallb4U, GRFFE.smallbsquared,GRHD.u4U_ito_ValenciavU)
GRHD.compute_g4DD_zerotimederiv_dD(gammaDD,betaU,alpha, metricderivDDD,shiftderivUD,lapsederivD)
GRHD.compute_S_tilde_source_termD(alpha, GRHD.sqrtgammaDET,GRHD.g4DD_zerotimederiv_dD, GRFFE.TEM4UU)
for i in range(3):
    desc = "Adds the source term to StildeD"+str(i)+"."
    name = "calculate_StildeD"+str(i)+"_source_term"
    outCfunction(
        outfile  = os.path.join(out_dir,subdir,name+".h"), desc=desc, name=name,
        params   ="const paramstruct *params,const REAL *auxevol_gfs, REAL *rhs_gfs",
        body     = general_access \
                  +metric_deriv_access[i]\
                  +outputC(GRHD.S_tilde_source_termD[i],"Stilde_rhsD"+str(i),"returnstring",params=outCparams).replace("IDX4","IDX4S")\
                  +write_final_quantity[i],
        loopopts ="InteriorPoints",
        rel_path_for_Cparams=os.path.join("../"))


NameError: name 'gammaDD' is not defined

We will now create the structures we need: register the gridfunctions `AD` and `BU`, declare `AD_dD` for $A_{i,j}$ and zero `BU` for $B^i$. Then, we'll build the standard form of `BU` that we will use: $$B^i = \epsilon^{ijk} A_{k,j}.$$

<a id='code_validation'></a>

# Step 3: Code Validation against original C code \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are stored in `GiRaFFE_NRPy/GiRaFFE_Ccode_library` or generated by `GiRaFFE_NRPy_A2B.py`


In [ ]:
gammaDD = ixp.register_gridfunctions_for_single_rank2("auxevol","gammaDD","sym01",DIM=3)
# Define the directory that we wish to validate against:
valdir = os.path.join("GiRaFFE_NRPy/GiRaFFE_Ccode_library/RHSs/")

# import GiRaFFE_NRPy.GiRaFFE_NRPy_A2B as A2B
# A2B.GiRaFFE_NRPy_A2B(valdir,gammaDD,AD,BU)

# import difflib
# import sys

# print("Printing difference between original C code and this code...")
# # Open the files to compare
# file = "driver_AtoB.h"

# print("Checking file " + file)
# with open(os.path.join(valdir,file)) as file1, open(os.path.join(outdir,file)) as file2:
#     # Read the lines of each file
#     file1_lines = file1.readlines()
#     file2_lines = file2.readlines()
#     num_diffs = 0
#     for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(outdir+file)):
#         sys.stdout.writelines(line)
#         num_diffs = num_diffs + 1
#     if num_diffs == 0:
#         print("No difference. TEST PASSED!")
#     else:
#         print("ERROR: Disagreement found with .py file. See differences above.")

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy_C_code_library-A2B](TTutorial-GiRaFFE_NRPy_C_code_library-A2B.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy-A2B")